In [1]:
import pandas as pd
import numpy as np

In [ ]:
# ImVisible

test = pd.read_csv('../data/Annotations/testing_file.csv', header=0, delimiter=',')
train = pd.read_csv('../data/Annotations/training_file.csv', header=0, delimiter=',')
validation = pd.read_csv('../data/Annotations/validation_file.csv', header=0, delimiter=',')

imvisible = pd.concat([test, train, validation])

In [ ]:
imvisible = imvisible.rename(columns={'block': 'blocked'})
imvisible["blocked"] = (imvisible["blocked"] == "blocked").astype(int)

imvisible["theta_rad"] = np.atan((imvisible["x2"]-imvisible["x1"])/(imvisible["y2"]-imvisible["y1"]))
imvisible["theta_deg"] = imvisible["theta_rad"] * 180/np.pi

orig_width = 4032
orig_height = 3024

new_width = 876
new_height = 657

y1_bt = imvisible["y1"] > imvisible["y2"]

imvisible["x1"] = (imvisible["x1"] / orig_width * new_width).round().astype(int)
imvisible["x2"] = (imvisible["x2"] / orig_width * new_width).round().astype(int)
imvisible["y1"] = (imvisible["y1"] / orig_height * new_height).round().astype(int)
imvisible["y2"] = (imvisible["y2"] / orig_height * new_height).round().astype(int)

imvisible["x"] = np.where(y1_bt, imvisible["x1"], imvisible["x2"])
imvisible["y"] = np.where(y1_bt, imvisible["y1"], imvisible["y2"])

imvisible["file"] = "/pedestrian-traffic-lights/"+imvisible["file"]
imvisible["zebra"] = 1

imvisible = imvisible[['file', 'zebra', 'mode', 'blocked', 'x', 'y', 'theta_rad', 'theta_deg']]

In [ ]:
# Global Streetscapes

global_street = pd.read_csv('../data/global-streetscapes.csv', header=0, delimiter=',')

global_street = global_street.rename(columns={'uuid': 'file'})
global_street["file"] = "/global-streetscapes/"+global_street["file"]+".jpeg"

global_street["zebra"] = 0
columns = ['mode', 'blocked', 'x', 'y', 'theta_rad', 'theta_deg']
for col in columns:
    global_street[col] = pd.NA

global_street = global_street[['file', 'zebra', 'mode', 'blocked', 'x', 'y', 'theta_rad', 'theta_deg']]

In [ ]:
df = pd.concat([imvisible, global_street])
df.to_csv("../data/dataset.csv", index=False)

In [ ]:
df = pd.read_csv('../data/dataset.csv', header=0, delimiter=',')

In [ ]:
import os

# df = df[("../data" + df['file'].astype(str)).apply(os.path.exists)]
cond1 = df["mode"].isna()
cond2 = df["mode"].isin([1, 2])

df["mode"] = np.select([cond1, cond2], [np.nan, 1], default=0)
df.to_csv("../data/dataset.csv", index=False)